In [ ]:
import os
import numpy as np
import cv2
import pandas as pd
from tqdm import tqdm_notebook

In [ ]:
OUTPUT_FOLDER = '../data/train_images_64x128'
DATA_FOLDER = '../data'
HEIGHT = 137
WIDTH = 236
SIZE = 128

In [ ]:
def center_image(image):
    mid = (image.max() - image.min()) / 2
    mask = image > mid
    image = image[np.ix_(mask.any(1),mask.any(0))]
    new_image = np.zeros((137, 236), dtype=np.uint8)
    top_left = ((137 - image.shape[0]) // 2, (236 - image.shape[1]) // 2)
    new_image[top_left[0]:top_left[0] + image.shape[0], top_left[1]:top_left[1] + image.shape[1]] = image
    return cv2.resize(new_image, (128, 64))
    
def preprocessing(image):
    image = np.asarray(image, dtype=np.uint8)
    image = 255 - image
    image.resize((137, 236))
    image = center_image(image)
#     image = cv2.resize(image, (128, 64))
    return image

In [ ]:
data_files = os.listdir(DATA_FOLDER)
train_parquets = []
for file in data_files:
    if 'train_image_data' in file:
        train_parquets.append(file)

In [ ]:
for file in train_parquets:
    df = pd.read_parquet(os.path.join(DATA_FOLDER, file)) 
    data = df.iloc[:, 1:].values.reshape(-1, HEIGHT, WIDTH).astype(np.uint8)
    for i in tqdm_notebook(range(len(df)), desc='output_images'):
        image_id = df.iloc[i,0]
        img = data[i]
        img = preprocessing(img)
        output_path = os.path.join(OUTPUT_FOLDER, image_id + '.png')
        cv2.imwrite(output_path, img)